In [74]:
from torch import empty
import math
import torch

In [75]:
torch.set_grad_enabled(False)

In [55]:
def generate_data(n=1000):
    inputs = empty(n, 2)
    inputs = empty(n, 2).uniform_(0,1)
    labels = (inputs - 0.5).pow(2).sum(1)<1/(2*math.pi)
    return inputs, labels

In [70]:
def split_dataset(inputs, lables, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    train_len = math.floor(inputs.size()[0] * train_ratio)
    val_len = math.floor(inputs.size()[0] * val_ratio)
    test_len = inputs.size()[0]-train_len-val_len
    
    train_inputs = inputs.narrow(0, 0, train_len)
    train_labels = labels.narrow(0, 0, train_len)
    
    validation_inputs = inputs.narrow(0, train_len, val_len)
    validation_labels = labels.narrow(0, train_len, val_len)
    
    test_inputs = inputs.narrow(0, train_len+val_len, test_len)
    test_labels = labels.narrow(0, train_len+val_len, test_len)
    
    return train_inputs, train_labels, validation_inputs, validation_labels, test_inputs, test_labels

In [71]:
inputs, labels = generate_data(n=1000)
train_inputs, train_labels, validation_inputs, validation_labels, test_inputs, test_labels = \
split_dataset(inputs, labels, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2) 


In [83]:
class Module (object) :
    """
    Base class for other neural network modules to inherit from
    """
    
    def __init__(self):
        self._author = 'HB_FB'
    
    def forward ( self , input ) :
        """ `forward` should get for input, and returns, a tensor or a tuple of tensors """
        raise NotImplementedError
        
    def backward ( self , gradwrtoutput ) :
        """
        `backward` should get as input a tensor or a tuple of tensors containing the gradient of the loss 
        with respect to the module’s output, accumulate the gradient wrt the parameters, and return a 
        tensor or a tuple of tensors containing the gradient of the loss wrt the module’s input.
        """
        raise NotImplementedError
        
    def param ( self ) :
        """ 
        `param` should return a list of pairs, each composed of a parameter tensor, and a gradient tensor 
        of same size. This list should be empty for parameterless modules (e.g. activation functions). 
        """
        return []

In [ ]:
class Linear(Module):
    
    def __init__(self, input_dim, output_dim, std=1):
        super().__init__()
        self.weight = empty(output_dim, input_dim).normal_(0, std)
        self.bias = Tensor(output_dim).normal_(0, std)
        self.dw = empty(self.weight.size()).zero_()
        self.db = empty(self.bias.size()).zero_()
        self.x = 0
         
    def forward(self, input_):
        self.x = input_
        return self.weight.mv(self.x) + self.bias
    
    def backward(self, grdwrtoutput):
        self.dw.add_(grdwrtoutput.view(-1,1).mm(self.x.view(1,-1)))
        self.db.add_(grdwrtoutput)
        return self.weight.t().mv(grdwrtoutput)
    
    def param (self):
        return [(self.weight, self.dw), (self.bias, self.db)]

In [90]:
class ReLu(Module):
    
    def __init__(self):
        super().__init__()
        self.input_ = 0
        
    def forward(self, input_):
        self.input_ = input_
        result = self.input_ * (self.input_ > 0).float()
        return result
    
    def backward(self, grdwrtoutput):
        return grdwrtoutput * (self.input_ > 0).float()    

    def param (self):
        return [(None, None)]  

In [131]:
class Tanh(Module):
    
    def __init__(self):
        super().__init__()
        self.input_ = 0
        
    def forward(self, input_):
        self.input_ = input_
        tanh = 2/(1+(-2*input_).exp())-1
        return tanh
    
    def backward(self, grdwrtoutput):
        result = 4*((self.input_.exp() + (-self.input_).exp()).pow(-2)) * grdwrtoutput
        return result
    
    def param (self):
        return [(None, None)]

In [ ]:
class Sequential(Module):
    
    def __init__(self, *argv):
        super().__init__()
        self.modules = []
        args = list(args)[0]
        for module in argv:
            self.modules.append(module)
    
    def forward(self, input_):
        result = input_
        for module in self.modules:
            result = module.forward(result)
        return result
    
    def backward(self, grdwrtoutput):
        modules_re = self.modules[::-1]
        result = grdwrtoutput
        for module in modules_re:
            result = module.backward(result)
    
    def param ( self ) :
        parameters = []
        for module in self.modules:
            parameters.append(module.param())
        return parameters

In [ ]:
def loss(pred, label):
    return (pred - label.float()).pow(2).sum()

def dloss(pred,label):
    return 2*(pred - label.float())